<a href="https://colab.research.google.com/github/shramee/snark-py/blob/main/snark_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: install galois

!pip install galois


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 11.5 MB/s eta 0:00:00


## R1CS and witness

R1CS comprises of three matrices describing all constraints between all signals (values). Mapping relationship between inputs, outputs and intermediate values.

Three matrices are `L`, `R` and `O` such that with witness `w`,

```
Lw  *  Rw  =  Ow
```

In [4]:
import galois
import numpy as np

p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
FP = galois.GF(p)

# input arguments
x = FP(2)
y = FP(3)

v1 = x * x
v2 = y * y
v3 = 5 * x * v1
v4 = 4 * v1 * v2
out = 5*x**3 - 4*x**2*y**2 + 13*x*y**2 + x**2 - 10*y

w = FP([1, out, x, y, v1, v2, v3, v4])

print("w\n", w)

R = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 5, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 4, 0, 0, 0],
         [0, 0, 13, 0, 0, 0, 0, 0]])

L = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0]])

O = FP([[0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 1],
         [0, 1, 0, 10, FP(p - 1), 0, FP(p - 1), 1]])

Lw = np.dot(L, w)
Rw = np.dot(R, w)
Ow = np.dot(O, w)

print("Lw\n", Lw)
print("Rw\n", Rw)

LwRw = np.multiply(Lw, Rw)

print("Lw * Rw\n", LwRw)

print("Ow\n", Ow)

assert np.all(LwRw == Ow)

w
 [  1 104   2   3   4   9  40 144]
Lw
 [2 3 4 9 9]
Rw
 [ 2  3 10 16 26]
Lw * Rw
 [  4   9  40 144 234]
Ow
 [  4   9  40 144 234]
